In [1]:
import environments
import numpy as np
from collections import deque
from collections import defaultdict
import matplotlib.pyplot as plt
from replaybuffer import *

from library import *
 
%matplotlib inline

In [2]:
env1 = environments.environment_gym('Acrobot-v1')
env = environments.env_wrapper(env1)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [3]:
env

In [20]:
state = env.reset()

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [43]:
class LinearModel(nn.Module):
    def __init__(self, state_size, action_size):
        super(LinearModel, self).__init__()
        self.linear = nn.Linear(state_size, action_size)

    def forward(self,x):
        out = self.linear(x)
        values, action = F.softmax(out).max(0)
        return action , F.log_softmax(out)[action]

In [44]:
model = LinearModel(env.env.observation_space().shape[0],env.action_size())
model(torch.tensor(env.reset()).float())

(tensor(0), tensor(-0.9354, grad_fn=<SelectBackward>))

In [51]:
trajectory_length = 1000
sample_size = 100

def generate_trajectory(env, model, sample_size):
    sample = []
    state = env.reset()
    while len(sample) != sample_size:
        action, log_prob = model(torch.tensor(state).float())
        new_state, reward, done, _ = env.step(action.item())
        sample.append((state, action, log_prob, reward, int(done)))
        if done:
            state = env.reset()
        else:
            state = new_state

    return sample

traj = generate_trajectory(env, model, 10)

In [52]:
traj[0]

(array([ 0.99705704,  0.07666334,  0.99904004,  0.04380641,  0.00749738,
        -0.08939245]),
 tensor(0),
 tensor(-0.9326, grad_fn=<SelectBackward>),
 -1.0,
 0)